In [56]:
from model import mgn
import torch
import yaml
# Định nghĩa các tham số của model
# args =  config_str
  # Tham số của model (nếu có)

# Khởi tạo model
# num_classes = 751
# config_file = '/home/linhphuong/Documents/documents/persion_re_id/MGN-pytorch-modify/config.yml'
# with open(config_file, 'r') as cf:
#     config_str = "\n" + cf.read()

# args = yaml.safe_load(config_str)
# args = config_str
# num_classes = args['num_classes']
# model = mgn.MGN(args)
class ConfigObject:
    pass

# Đường dẫn đến tệp YAML
yaml_file_path = '/home/linhphuong/Documents/documents/persion_re_id/MGN-pytorch-modify/config.yml'

# Đọc tệp YAML
with open(yaml_file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

# Tạo một đối tượng từ dữ liệu YAML
config = ConfigObject()
for key, value in yaml_data.items():
    setattr(config, key, value)
model = mgn.MGN(config)
print(model)
# Đường dẫn đến file chứa trọng số đã lưu
path_to_weights = '/home/linhphuong/Documents/documents/persion_re_id/MGN-pytorch-modify/model_2.pth'

model_load = torch.load(path_to_weights)
for key in list(model_load.keys()):
    new_key = key.replace('model.', '')
    model_load[new_key] = model_load.pop(key)
# Tải trọng số đã lưu
model.load_state_dict(model_load)

# Chuyển model sang chế độ đánh giá (evaluation mode)
model.eval()

# Sử dụng model để thực hiện dự đoán trên dữ liệu mới (nếu cần)


/home/linhphuong/anaconda3/envs/id/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/linhphuong/anaconda3/envs/id/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MGN(
  (backone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256,

MGN(
  (backone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256,

In [57]:
import os
import torch
import numpy as np
import utils.utility as utility
from scipy.spatial.distance import cdist
from utils.functions import cmc, mean_ap
from utils.re_ranking import re_ranking

In [58]:
import trainer

In [59]:
import torchvision.transforms as T
def extract_transforms(input_image):
    transform = T.Compose([
        T.Resize((384,128), interpolation=3),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(input_image)

In [60]:
from PIL import Image
from matplotlib import pyplot as plt
import matplotlib
image_path = "/home/linhphuong/Documents/documents/persion_re_id/MGN-pytorch-modify/data/Market-1501-v15.09.15/query/0005_c2s1_000976_00.jpg"

# Open the image
image = Image.open(image_path)

# plt.imshow(image)

In [61]:
img = extract_transforms(image).unsqueeze(0)

In [84]:
model(img)

(tensor([[0.7722, 0.4122, 1.3220,  ..., 0.5726, 0.0479, 0.5804]],
        grad_fn=<CatBackward0>),
 tensor([[0.7722, 0.4122, 1.3220, 0.0000, 0.0000, 0.6904, 0.4226, 0.0000, 0.3096,
          0.0000, 0.0681, 0.5600, 0.0000, 0.4786, 0.0403, 0.2694, 0.0000, 0.0000,
          0.0000, 0.5123, 0.4989, 0.0000, 0.0000, 0.0000, 2.5041, 0.0000, 1.3342,
          0.8394, 1.4132, 0.5977, 0.0000, 0.0000, 0.0000, 1.7530, 0.0000, 0.0000,
          0.0000, 0.0164, 0.0000, 0.7505, 0.6525, 0.0000, 0.0000, 0.0000, 1.1747,
          0.0000, 0.0000, 0.8754, 1.1577, 0.0000, 0.0000, 0.0000, 0.8064, 0.3281,
          0.1120, 0.0000, 1.6208, 0.6309, 0.0461, 0.7399, 0.0000, 0.0000, 1.1718,
          0.0000, 0.2031, 0.0000, 0.0000, 0.1111, 0.0000, 1.3140, 0.0000, 0.0000,
          0.2820, 0.4093, 0.0000, 0.0000, 0.2870, 1.6629, 1.7339, 0.0000, 0.0000,
          0.0000, 1.2726, 0.2359, 0.1444, 0.0502, 1.9064, 0.0000, 0.7665, 0.4008,
          0.3255, 0.0000, 0.2079, 0.0000, 0.0000, 0.0000, 0.9305, 0.0000, 1.3412,

In [89]:
def fliphor(inputs):
    inv_idx = torch.arange(inputs.size(3)-1,-1,-1).long()  # N x C x H x W
        # torch.save(self.model.state_dict(), "./model_4.pth")
    return inputs.index_select(3,inv_idx)

def extract_feature(model,input):
        features = torch.FloatTensor()
        # for (inputs, labels) in loader:
        ff = torch.FloatTensor(input.size(0), 2048).zero_()
        for i in range(2):
            if i==1:
                input = fliphor(input)
            # input_img = inputs.to(self.device)
            outputs = model(input)
            f = outputs[0].data.cpu()
            ff = ff + f
            # print(len(outputs))
        fnorm = torch.norm(ff, p=2, dim=1, keepdim=True)
        
        ff = ff.div(fnorm.expand_as(ff))

        features = torch.cat((features, ff), 0)
        # torch.save(self.model.state_dict(), "./model_5.pth")

        return features

In [90]:
e_f = extract_feature(model,img)

12
12


In [64]:
e_f.shape

torch.Size([1, 2048])

In [65]:
from data.common import list_pictures

from torch.utils.data import dataset
from torchvision.datasets.folder import default_loader
class Market1501(dataset.Dataset):
    def __init__(self, args, transform, dtype):

        self.transform = transform
        self.loader = default_loader

        data_path = args.datadir
        if dtype == 'train':
            data_path += '/bounding_box_train'
        elif dtype == 'test':
            data_path += '/bounding_box_test'
        else:
            data_path += '/query'

        
        self.imgs = [path for path in list_pictures(data_path) if self.id(path) != -1]

        self._id2label = {_id: idx for idx, _id in enumerate(self.unique_ids)}

    def __getitem__(self, index):
        path = self.imgs[index]
        target = self._id2label[self.id(path)]

        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)

        return img, target

In [66]:
import os
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.file_list = os.listdir(root_dir)
        self.transform = T.Compose([
            T.Resize((384,128), interpolation=3),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.file_list[idx])
        image = Image.open(img_name)
        image = self.transform(image)

        # Extract label from the first 4 characters of the image name
        label = int(img_name.split(os.path.sep)[-1][:4])

        return image, label

In [67]:
# Specify the path to the dataset
dataset_path = "/home/linhphuong/Documents/documents/persion_re_id/MGN-pytorch-modify/data/Market-1501-v15.09.15/test"

# Create a CustomDataset instance
custom_dataset = CustomDataset(root_dir=dataset_path)

# Create a DataLoader for the dataset
batch_size = 4
data_loader = DataLoader(dataset=custom_dataset, batch_size=batch_size, shuffle=True)

In [92]:
import torch
import pandas as pd

# Create empty lists to store the embeddings and labels
embedding_list = []
label_list = []

# Iterate over the batches of data
for batch_images, batch_labels in data_loader:
    # Move the batch of images to the device (GPU if available)
    # batch_images = batch_images.to(device)
    
    # Pass the batch of images through the model

    output = extract_feature(model,batch_images)
    # output = torch.tensor(output)
    output = list(output)
    # print("output", output)
    # Split the output embeddings for each image in the batch
    # embeddings = torch.split(output, 1)
    
    # Iterate over the embeddings and labels in the batch
    for embedding, label in zip(output, batch_labels):
        # Convert the embedding tensor to a list of numbers
        embedding_list.append(embedding.squeeze().cpu().detach().numpy().tolist())
        label_list.append(label.item())

12
12


In [93]:
data = [embedding + [label] for embedding, label in zip(embedding_list, label_list)]
print(len(data))
# Define column names
columns = [f"feature_{i}" for i in range(len(embedding_list[0]))] + ["Label"]

# Create DataFrame from the list of lists
df = pd.DataFrame(data, columns=columns)
# Save the DataFrame to a CSV file
df.to_csv('/home/linhphuong/Documents/documents/persion_re_id/MGN-pytorch-modify/embeddings.csv', index=False)

4


In [94]:
# Specify the path to the dataset
dataset_path2 = "/home/linhphuong/Documents/documents/persion_re_id/MGN-pytorch-modify/data/Market-1501-v15.09.15/test"

# Create a CustomDataset instance
custom_dataset2 = CustomDataset(root_dir=dataset_path2)

# Create a DataLoader for the dataset
batch_size = 4
data_loader2 = DataLoader(dataset=custom_dataset2, batch_size=batch_size, shuffle=True)

In [100]:
import torch
import pandas as pd

# Create empty lists to store the embeddings and labels
embedding_list2 = []
label_list2 = []

# Iterate over the batches of data
for batch_images, batch_labels in data_loader2:
    # Move the batch of images to the device (GPU if available)
    # batch_images = batch_images.to(device)
    
    # Pass the batch of images through the model
    output = extract_feature(model,batch_images)
    embeddings = list(output)
    # Split the output embeddings for each image in the batch
    # embeddings = torch.split(output, 1)
    
    # Iterate over the embeddings and labels in the batch
    for embedding, label in zip(embeddings, batch_labels):
        # Convert the embedding tensor to a list of numbers
        embedding_list2.append(embedding.squeeze().cpu().detach().numpy().tolist())
        label_list2.append(label.item())

12
12


In [101]:
# Combine embeddings and labels into a list of lists
data2 = [embedding + [label] for embedding, label in zip(embedding_list2, label_list2)]

# Define column names
columns2 = [f"feature_{i}" for i in range(len(embedding_list2[0]))] + ["Label"]

# Create DataFrame from the list of lists
df2 = pd.DataFrame(data2, columns=columns2)

# Save the DataFrame to a CSV file
df2.to_csv('/home/linhphuong/Documents/documents/persion_re_id/MGN-pytorch-modify/embeddings_test.csv', index=False)

In [102]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('/home/linhphuong/Documents/documents/persion_re_id/MGN-pytorch-modify/embeddings.csv')

# Split the DataFrame into features (embeddings) and labels
X_train = df.iloc[:, :-1]
y_train = df.iloc[:, -1]

# Create a KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
print(df)
# Fit the classifier to the training data
knn.fit(X_train, y_train)

   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0   0.024536   0.010658   0.068860        0.0   0.005983   0.040559   
1   0.018952   0.000385   0.047369        0.0   0.001945   0.006917   
2   0.000000   0.000000   0.056089        0.0   0.027257   0.029233   
3   0.041531   0.000000   0.042104        0.0   0.033183   0.040782   

   feature_6  feature_7  feature_8  feature_9  ...  feature_2039  \
0   0.043827   0.000000   0.058975        0.0  ...      0.000000   
1   0.021748   0.000000   0.034938        0.0  ...      0.004057   
2   0.028987   0.000000   0.031728        0.0  ...      0.004930   
3   0.053861   0.031068   0.013767        0.0  ...      0.047855   

   feature_2040  feature_2041  feature_2042  feature_2043  feature_2044  \
0           0.0      0.000000      0.007931      0.010375           0.0   
1           0.0      0.000863      0.000375      0.022104           0.0   
2           0.0      0.012133      0.000000      0.029638           0.0   
3  

KNeighborsClassifier(n_neighbors=3)

In [103]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Read the CSV file into a pandas DataFrame
df_test = pd.read_csv('/home/linhphuong/Documents/documents/persion_re_id/MGN-pytorch-modify/embeddings_test.csv')

# Split the DataFrame into features (embeddings) and labels
X_test = df_test.iloc[:, :-1]
y_test = df_test.iloc[:, -1]

# Ensure that X_test has the same format as X_train
# If X_train has more features, you can adjust X_test accordingly

# Use the trained KNN classifier to predict the labels for the test data
y_pred = knn.predict(X_test.values)

# Calculate the accuracy of the predictions
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print("Accuracy:", accuracy)

/home/linhphuong/anaconda3/envs/id/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


Accuracy: 0.75
